# Voice Identification

### n-shot learning

The idea of one shot learning is to train an algorithm so that after only `n` examples, an algorithm can identify a example again. 

Siamese networks attempt to do this by training a model to learn how to take a high dimensional input an generate a feature vector. The network is trained by taking two examples either similar or different and training them to reduce/increase the distance of the feature vector generated by the output.

### Applying to Voice Identification

The end goal of this model is to take a audio sample that has undergone speech diarihsation and identify each speaker in the set.

### Datasets

| Name | Speakers           | Min  | Max
| ------------- |-------------|-----|-----|
[VoxCeleb](http://www.robots.ox.ac.uk/~vgg/data/voxceleb/)| 7000+| 3s | 3s
[10 English Speakers](http://www.openslr.org/resources/45/ST-AEDS-20180100_1-OS) | 10 | ? | ? 

### Articles

- https://github.com/zdmc23/oneshot-audio/blob/master/OneShot.ipynb

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import sys
sys.path.append("../audio")
from audio import *
from siamese import *
np.random.seed(2)

In [8]:
path = Config.data_path() / 'voxceleb-small'

In [14]:
config = AudioTransformConfig()
config.segment_size = 1000
# config.remove_silence = True
config.f_max = 8000
config.to_db_scale = True
config.top_db = 80
# config.silence_threshold = 30
config.max_to_pad = 4000
# config.silence_padding = 300
label_pattern = r'/(id(\d+))/'
audios = AudioList.from_folder(path, recurse=True, config=config).split_none().label_from_re(label_pattern)
audios.c

66

In [15]:
sds = SiameseDataset.create_from_ll(audios, split_c=.2, tar_num=10000)

(86972, 86972)


RuntimeError: [enforce fail at CPUAllocator.cpp:56] posix_memalign(&data, gAlignment, nbytes) == 0. 12 vs 0
